# DAT210x - Programming with Python for DS

## Module6- Lab1

In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np 
import time

Feel free to adjust and experiment with these parameters after you have completed the lab:

In [10]:
C = 1
kernel = 'linear'

In [11]:
# TODO: Change to 200000 once you get to Question#2
iterations = 5000

# You can set this to false if you want to draw the full square matrix:
FAST_DRAW = True

### Convenience Functions

In [7]:
def drawPlots(model, X_train, X_test, y_train, y_test, wintitle='Figure 1'):
    # You can use this to break any higher-dimensional space down,
    # And view cross sections of it.

    # If this line throws an error, use plt.style.use('ggplot') instead
    mpl.style.use('ggplot') # Look Pretty

    padding = 3
    resolution = 0.5
    max_2d_score = 0

    y_colors = ['#ff0000', '#00ff00', '#0000ff']
    my_cmap  = mpl.colors.ListedColormap(['#ffaaaa', '#aaffaa', '#aaaaff'])
    colors   = [y_colors[i] for i in y_train]
    num_columns = len(X_train.columns)

    fig = plt.figure()
    fig.canvas.set_window_title(wintitle)
    fig.set_tight_layout(True)
    
    cnt = 0
    for col in range(num_columns):
        for row in range(num_columns):
            
            # Easy out
            if FAST_DRAW and col > row:
                cnt += 1
                continue

            ax = plt.subplot(num_columns, num_columns, cnt + 1)
            plt.xticks(())
            plt.yticks(())

            # Intersection:
            if col == row:
                plt.text(0.5, 0.5, X_train.columns[row], verticalalignment='center', horizontalalignment='center', fontsize=12)
                cnt += 1
                continue


            # Only select two features to display, then train the model
            X_train_bag = X_train.ix[:, [row,col]]
            X_test_bag = X_test.ix[:, [row,col]]
            model.fit(X_train_bag, y_train)

            # Create a mesh to plot in
            x_min, x_max = X_train_bag.ix[:, 0].min() - padding, X_train_bag.ix[:, 0].max() + padding
            y_min, y_max = X_train_bag.ix[:, 1].min() - padding, X_train_bag.ix[:, 1].max() + padding
            xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution),
                                 np.arange(y_min, y_max, resolution))

            # Plot Boundaries
            plt.xlim(xx.min(), xx.max())
            plt.ylim(yy.min(), yy.max())

            # Prepare the contour
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            plt.contourf(xx, yy, Z, cmap=my_cmap, alpha=0.8)
            plt.scatter(X_train_bag.ix[:, 0], X_train_bag.ix[:, 1], c=colors, alpha=0.5)


            score = round(model.score(X_test_bag, y_test) * 100, 3)
            plt.text(0.5, 0, "Score: {0}".format(score), transform = ax.transAxes, horizontalalignment='center', fontsize=8)
            max_2d_score = score if score > max_2d_score else max_2d_score

            cnt += 1

    print("Max 2D Score: ", max_2d_score)

In [6]:
def benchmark(model, X_train, X_test, y_train, y_test, wintitle='Figure 1'):
    print(wintitle + ' Results')
    s = time.time()
    
    for i in range(iterations):
        # TODO: train the classifier on the training data / labels:
        
        # .. your code here ..
        model.fit(X_train,y_train)
        
    print("{0} Iterations Training Time: ".format(iterations), time.time() - s)


    s = time.time()
    for i in range(iterations):
        # TODO: score the classifier on the testing data / labels:

        # .. your code here ..
        score=model.score(X_test,y_test)
        
    print("{0} Iterations Scoring Time: ".format(iterations), time.time() - s)
    print("High-Dimensionality Score: ", round((score*100), 3))

### The Assignment

Load up the wheat dataset into dataframe `X` and verify you did it properly. Indices shouldn't be doubled, nor should you have any headers with weird characters...

In [164]:
# .. your code here ..
X=pd.read_csv('Datasets/wheat.data')
X=X.drop(labels=['id'],axis=1)


In [165]:
# An easy way to show which rows have nans in them:
X[pd.isnull(X).any(axis=1)]




,area,perimeter,compactness,length,width,asymmetry,groove,wheat_type
7,14.11,14.10,0.8911,5.4200,3.302,2.700,NaN,canadian
35,16.12,15.00,NaN,0.9000,NaN,5.709,3.485,canadian
60,11.42,12.86,0.8683,5.0080,2.850,2.700,NaN,canadian
135,15.38,14.66,0.8990,5.4770,3.465,3.600,NaN,canadian
169,11.24,13.00,NaN,0.8359,5.090,2.715,3.521,canadian
170,11.02,13.00,NaN,0.8189,5.325,2.701,6.735,canadian
201,12.67,13.32,0.8977,4.9840,3.135,2.300,NaN,canadian


Go ahead and drop any row with a nan:

In [166]:
# .. your code here ..
X.dropna(axis=1,inplace=True)
#X[pd.isnull(X).any(axis=1)]
X.head()

,area,perimeter,length,asymmetry,wheat_type
0,15.26,14.84,5.763,2.221,kama
1,14.88,14.57,5.554,1.018,kama
2,14.29,14.09,5.291,2.699,kama
3,13.84,13.94,5.324,2.259,kama
4,16.14,14.99,5.658,1.355,kama


In the future, you might try setting the nan values to the mean value of that column, the mean should only be calculated for the specific class rather than across all classes, now that you have the labels.

Copy the labels out of the dataframe into variable `y`, then remove them from `X`.

Encode the labels, using the `.map()` trick we showed you in Module 5, such that `canadian:0`, `kama:1`, and `rosa:2`.

In [167]:
# .. your code here..
y=X[['wheat_type']]
y.head()
X.drop('wheat_type',axis=1,inplace=True)
X.drop('asymmetry',axis=1,inplace=True)

y=y['wheat_type'].map({'canadian':0,'kama':1,'rosa':2})
#y[pd.isnull(y).any(axis=0)]
#y.dropna(axis=1,inplace=True)
#X.head()
#X.remove(y)
#X=X.astype('wheat_type').cat.codes

Split your data into a `test` and `train` set. Your `test` size should be 30% with `random_state` 7. Please use variable names: `X_train`, `X_test`, `y_train`, and `y_test`:

In [168]:
# .. your code here ..
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.30, random_state=7)
#y_train
#X_train

Create an SVC classifier named `svc` and use a linear kernel. You already have `C` defined at the top of the lab, so just set `C=C`.

In [169]:
# .. your code here ..
from sklearn.svm import SVC
svc= SVC(kernel='linear',C=C)
#model.fit(X, y)

Create an KNeighbors classifier named `knn` and set the neighbor count to `5`:

In [170]:
# .. your code here ..
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
#model.fit(X_train, y_train) 

### Fire it Up:

In [171]:
benchmark(knn, X_train, X_test, y_train, y_test, 'KNeighbors')
drawPlots(knn, X_train, X_test, y_train, y_test, 'KNeighbors')

KNeighbors Results
('200000 Iterations Training Time: ', 108.60699987411499)
('200000 Iterations Scoring Time: ', 194.51400017738342)
('High-Dimensionality Score: ', 87.302)
('Max 2D Score: ', 90.476)


In [172]:
benchmark(svc, X_train, X_test, y_train, y_test, 'SVC')
drawPlots(svc, X_train, X_test, y_train, y_test, 'SVC')

SVC Results
('200000 Iterations Training Time: ', 147.0789999961853)
('200000 Iterations Scoring Time: ', 89.40700006484985)
('High-Dimensionality Score: ', 90.476)
('Max 2D Score: ', 92.063)


In [146]:
plt.show()

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
X=pd.read_csv('Datasets/wheat.data')
X=X.drop(labels=['id'],axis=1)
X.dropna(axis=0,inplace=True)
y=X[['wheat_type']]
X.drop('wheat_type',axis=1,inplace=True)
#X.drop('asymmetry',axis=1,inplace=True)
y=y['wheat_type'].map({'canadian':0,'kama':1,'rosa':2})
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.30, random_state=7)

model = tree.DecisionTreeClassifier(max_depth=1,random_state=2)
#, criterion="entropy")

#svc= SVC(kernel='linear',C=1)

knn = KNeighborsClassifier(n_neighbors=5)

benchmark(model, X_train, X_test, y_train, y_test, 'DecisionTree')
drawPlots(model, X_train, X_test, y_train, y_test, 'DecisionTree')

benchmark(knn, X_train, X_test, y_train, y_test, 'KNeighbors')
drawPlots(knn, X_train, X_test, y_train, y_test, 'KNeighbors')

#benchmark(svc, X_train, X_test, y_train, y_test, 'SVC')
#drawPlots(svc, X_train, X_test, y_train, y_test, 'SVC')

DecisionTree Results
('5000 Iterations Training Time: ', 2.619000196456909)
('5000 Iterations Scoring Time: ', 1.681999921798706)
('High-Dimensionality Score: ', 68.852)
('Max 2D Score: ', 68.852)
KNeighbors Results
('5000 Iterations Training Time: ', 2.7660000324249268)
('5000 Iterations Scoring Time: ', 5.468999862670898)
('High-Dimensionality Score: ', 83.607)
('Max 2D Score: ', 90.164)


### Bonus:

After submitting your answers, mess around with the gamma, kernel, and C values.